In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm, trange 
import os
import glob
import traceback
import datetime
import warnings
warnings.filterwarnings('ignore')


# to display maximum rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [2]:
# file paths
output_path = r"D:\Sourav\Stock-EOD-05-95DeltaStrikesDividendStocks\\"
underlying_path = r"E:\sourav\strikeSelector\strikeSelector\input\5Years\EQTill04Jul2022\AllSymbols\\"
dispersion_path = r"D:\Sourav\Stocks-EOD-GreeksDividendStocks\\"
equity_path = r"E:\sourav\strikeSelector\strikeSelector\input\5Years\EQTill04Jul2022\AllSymbols\\"

# select delta values
delta_list = [0.05, 0.10, 0.15, 0.20, 0.25,
              0.30, 0.35, 0.40, 0.45, 0.50, 
              0.55, 0.60, 0.65, 0.70, 0.75, 
              0.80, 0.85, 0.90, 0.95]

all_files = sorted(next(os.walk(dispersion_path))[2])
all_underlying_files = os.listdir(underlying_path)
print(len(all_files))
print(len(all_underlying_files))

80
1047


In [1]:
# files to be ignored
#ignore_files = ['AUBANK-I.csv', 'AUBANK-II.csv', 'AUBANK-III.csv']
#all_files = sorted(list(set(all_files) - set(ignore_files)))

symbol = 'IOC'
all_files = [f'{symbol}-I.csv', f'{symbol}-II.csv', f'{symbol}-III.csv']
print(len(all_files))

# symbols = next(os.walk(output_path))[2]
# print(len(symbols))
# all_files = np.setdiff1d(all_files, symbols)
print((all_files))
#print(len(all_underlying_files))

3
['IOC-I.csv', 'IOC-II.csv', 'IOC-III.csv']


In [9]:
print(datetime.datetime.now())
filesProcessed = 0
filesNotProcessed = 0
notProcessedFiles = []

# load main csv
for filename in tqdm(all_files):
    print('Start time : ', datetime.datetime.now())
    print(dispersion_path + filename)
    df = pd.read_csv(dispersion_path + filename)
    print(df.shape[0])
    df = df.rename(columns={'tickername' : 'Ticker'})
#     df.drop(columns=df.columns[0], axis=1, inplace=True)
      
    
    # remove 'Unnamed' columns in a csv
    remove_cols = [col for col in df.columns if 'Unnamed' in col]
    df.drop(remove_cols, axis='columns', inplace=True)   

    #print(df.iloc[0:500])
    print('filename :', filename)
    
    # change date column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

    # replace $ character in a ticker with empty string ('')
    df['Ticker'].replace({'$': None},inplace =True)
    
    
    #df = df.drop(['Strike'], axis=1)
    
    df.rename(columns={
                       'Adj_Open' : 'OPT_Open',
                       'Adj_High' : 'OPT_High',
                       'Adj_Low' : 'OPT_Low',
                       'Adj_Close' : 'OPT_Close',
                       'Adj_Volume' : 'OPT_Contracts',
                       'Volume' : 'OPT_Contracts',
                        #'New_OI' : 'OPT_OI',
                       'Last_OI' : 'OPT_OI',
                       'OpenInterest' : 'OPT_OI',
                       'Final_strike' : 'Strike',
                       'CONTRACTS' : 'OPT_Contracts',
                       'OPEN_INT' : 'OPT_OI'}, inplace=True)
    df2 = df.copy()
    

    # extract Strike and Option Type from the Ticker if it's not there in the input file already.
    
#     df['Strike'] = df['Ticker'].str.extract('([0-9]+[./]*[0-9]*)').astype(float)
    
#     df['Option_Type'] = df["Ticker"].str.split('-').str[0].str[-2:]

#     df['Option_Type'] = np.where((df['Option_Type'] == 'CE') | (df['Option_Type'] == 'PE'),
#                                  df['Option_Type'], 'XX')

    
    # get the symbol from filename
    symbolFilename = filename.split('.')[0]
    print('symbolFilename :', symbolFilename)
    
    # remove '-I'/'-II'/'-III' from filename
    symbol = symbolFilename.replace('-III', '')
    symbol = symbol.replace('-VIII', '')
    symbol = symbol.replace('-VII', '')
    symbol = symbol.replace('-VI', '')
    symbol = symbol.replace('-V', '')
    symbol = symbol.replace('-IV', '')
    symbol = symbol.replace('-II', '')
    symbol = symbol.replace('-I', '')
    print('symbol :', symbol)
    
    # substitute any special characters ('-', '&', '_') in Dispersion file names with empty string ('_')
    charactersDroppedSymbol = re.sub('\ |\_|\.|\-|\&|\;|\:', '', symbol)

    df2 = df.copy()
    
    ''' 
    for eqFilename in all_underlying_files:
    
        # replace any special characters ('-', '&', '_') in EQ file names with empty string ('')
        charactersDroppedEQFilename = re.sub('\_|\-|\&', '', eqFilename.split('.')[0]).replace('$', '')
        
#         print('charactersDroppedEQFilename :', charactersDroppedEQFilename)
#         print('charactersDroppedSymbol :', charactersDroppedSymbol)
        
        # check if symbol and EQ filenames are same after special characters being dropped 
        if (charactersDroppedEQFilename == charactersDroppedSymbol):
            
            print('eqFilename :', eqFilename)
            
            df_underlying = pd.read_csv(equity_path + eqFilename)
            
            df_underlying.rename(columns={'Open' : 'EQ_Open',
                                          'High' : 'EQ_High',
                                          'Low' : 'EQ_Low',
                                          'Close' : 'EQ_Close',
                                          'Volume' : 'EQ_Volume'}, inplace=True)
            df_underlying['Date'] = pd.to_datetime(df_underlying['Date'], dayfirst=True)
    
    # merge main csv and underlying csv
    try:
        df2 = df.merge(df_underlying[['Date', 'EQ_Open', 'EQ_High', 'EQ_Low', 'EQ_Close']], on=['Date'], how='left')
        df_dates = df['Date'].unique()
        df_underlying_dates = df_underlying['Date'].unique()
        diff_dates = np.setdiff1d(df_dates, df_underlying_dates)
        print('Dates missing in EQ file : ', diff_dates)
        df_underlying = pd.DataFrame()
        
    except Exception as e1:
        
        #print('e :', e1)
        print('Symbol not found in EQ folder : ', filename)
        notProcessedFiles.append(filename)
        filesNotProcessed += 1
        continue    
    '''

    # get the hyphen type (-I, -II, -III) for grouping based on current
    #df2['Ticker_Type'] = df2['Ticker'].str.split('-').str[1]
    
    
    #df2 = df2[df2['OPT_Contracts']>=1000]
    
    # calculate difference between 'EQ_Close' and 'Strike' and get the minimum difference for a group
    df2['Difference'] = abs(df2['EQ_Close'] - df2['Strike'].astype(float))
    dfg = df2.groupby(['Date', 'Option_Type'])['Difference']
    df2['Min'] = dfg.transform('min')
    
    # delete output file if it already exists
    try:
        os.remove(output_path + symbolFilename + '.csv')
    except Exception as e:
        #print('e1 : ', e)
        pass
    
    print(df2.shape[0])
    #print(df2.columns)
    
    # get 'At The Money' for each group
    dfg = df2.groupby(['Date', 'Option_Type'])
    for name, group in tqdm(dfg):
        
        # get 'At The Money' for each group
        try:
            atTheMoney = max(group[(group['Difference'] == group['Min'])]['Strike'])
            group['At_The_Money'] = atTheMoney
        
        except Exception as e:
            #print('e3 : ', e)
            group['At_The_Money'] = np.nan
    
        if symbol != 'BANKNIFTY':
            group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
                       'Strike', 'Option_Type', 'Expiry_Date', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
                       'EQ_High', 'EQ_Low', 'EQ_Close', 'At_The_Money']]
        else:
            group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
                       'Strike', 'Option_Type', 'Expiry_Date', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
                       'EQ_High', 'EQ_Low', 'EQ_Close', 'At_The_Money']]
        
        # loop through different delta values
        for delta in delta_list:
            
            # convert delta into string
            # delta = str(int(delta * 100))
            
            if delta == 0.50:
                group.rename(columns={'Min' : f'Delta_{delta*100:.0f}_Diff_Min',
                                      'Difference' : f'Delta_{delta*100:.0f}_Diff'}, 
                             inplace=True)
                group[f'Delta_{delta*100:.0f}_Strike'] = group['At_The_Money']
            else:
                if group['Option_Type'].iloc[0] == 'CE':
                    group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - delta)
                elif group['Option_Type'].iloc[0] == 'PE':
                    group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - (-delta))
                elif group['Option_Type'].iloc[0] == 'XX':
                    group[f'Delta_{delta*100:.0f}_Diff'] = np.nan

                group[f'Delta_{delta*100:.0f}_Diff_Min'] = group[f'Delta_{delta*100:.0f}_Diff'].min()        

                try:
                    deltaStrike = max(group[group[f'Delta_{delta*100:.0f}_Diff'] == group[f'Delta_{delta*100:.0f}_Diff_Min']]['Strike'])
                    group[f'Delta_{delta*100:.0f}_Strike'] = deltaStrike
                except Exception as e:
                    #print('e2 : ', e)
                    group[f'Delta_{delta*100:.0f}_Strike'] = np.nan

                # dropping unnecessary columns
                group = group.drop([f'Delta_{delta*100:.0f}_Diff_Min', f'Delta_{delta*100:.0f}_Diff'], axis=1)
                
#         select required columns in the output csv   
#         if symbol != 'BANKNIFTY':
#             group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
#                        'Strike', 'Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
#                        'EQ_High', 'EQ_Low', 'EQ_Close', 'EQ_Volume', 'At The Money', f'Delta_{delta*100:.0f}_Diff', f'Delta_{delta*100:.0f}_Diff_Min',
#                        f'Delta_{delta*100:.0f}_Strike', 'Ticker_Type']]
#         else:
#             group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
#                        'Strike', 'Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
#                        'EQ_High', 'EQ_Low', 'EQ_Close', 'At The Money', f'Delta_{delta*100:.0f}_Diff', f'Delta_{delta*100:.0f}_Diff_Min',
#                        f'Delta_{delta*100:.0f}_Strike', 'Ticker_Type']]

        # write output to csv
        group.to_csv(output_path + symbolFilename + '.csv', mode='a', header=not os.path.exists(output_path + symbolFilename + '.csv'), index=False)
    filesProcessed += 1
    print('End time : ', datetime.datetime.now())

print('Files processed :', filesProcessed)
print('Files not processed :', filesNotProcessed)  
print('Not processed files :', notProcessedFiles)
print(datetime.datetime.now())


2023-06-15 12:39:14.064035


  0%|          | 0/3 [00:00<?, ?it/s]

Start time :  2023-06-15 12:39:14.071503
D:\Sourav\Stocks-EOD-GreeksDividendStocks\\IOC-I.csv
69998
filename : IOC-I.csv
symbolFilename : IOC-I
symbol : IOC
69998


  0%|          | 0/5657 [00:00<?, ?it/s]

End time :  2023-06-15 12:43:19.132180
Start time :  2023-06-15 12:43:19.147772
D:\Sourav\Stocks-EOD-GreeksDividendStocks\\IOC-II.csv
19767
filename : IOC-II.csv
symbolFilename : IOC-II
symbol : IOC
19767


  0%|          | 0/3539 [00:00<?, ?it/s]

End time :  2023-06-15 12:45:47.579191
Start time :  2023-06-15 12:45:47.579191
D:\Sourav\Stocks-EOD-GreeksDividendStocks\\IOC-III.csv
430
filename : IOC-III.csv
symbolFilename : IOC-III
symbol : IOC
430


  0%|          | 0/321 [00:00<?, ?it/s]

End time :  2023-06-15 12:46:01.196044
Files processed : 3
Files not processed : 0
Not processed files : []
2023-06-15 12:46:01.196044


In [ ]:
df=pd.read_csv(r"E:\sourav\BANKNIFTYConstituents\BANKNIFTYMonthly\05-95DeltaStrikes\BANKNIFTY-I.csv")
df.shape[0]

## Append -I, -II and -III files 

In [ ]:
input_path = r"C:\Sourav\BANKNIFTYConstituents27-01-2023to15-03-2023\05-95DeltaStrikes\\"
output_path = r"C:\Sourav\BANKNIFTYConstituents27-01-2023to15-03-2023\Combined\\"
symbols = next(os.walk(input_path))[2]

for i in tqdm(range(len(symbols))):
    symbols[i] = symbols[i].replace('-III.csv', '').replace('-II.csv', '').replace('-I.csv', '')
    
symbols = sorted(set(symbols))
#symbols = ['ICICIBANK']
symbols

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    print(symbol)
    sum1 = 0
    for i in hyphens:
        print(i)
        file_path = input_path + symbol + i + '.csv'
        if os.path.isfile(file_path):
            df1 = pd.read_csv(file_path)
            #df = df.append(df1)
            df1.to_csv(output_path + symbol + '.csv', mode='a', header=not os.path.exists(output_path + symbol + '.csv'),
                       index=False)
        sum1 += df1.shape[0]
    print(sum1)
    

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    df = pd.DataFrame()
    sum1 = 0
    for i in hyphens:
        file_path = input_path + symbol + i + '.csv'
        if os.path.isfile(file_path):

In [ ]:
input_path = r"D:\Sourav\BN\05-50DeltaStrikesAdded\\"
new_close_path = r"D:\purvang\BN\New Close\\"
output_path = r"D:\Sourav\BN\05-50DeltaStrikesAddedFutMonthly\\"

symbols = next(os.walk(input_path))[2]
#symbols = ['AUBANK-I.csv', 'AUBANK-II.csv', 'AUBANK-III.csv']

for i in tqdm(range(len(symbols))):
    symbols[i] = symbols[i].replace('-III.csv', '').replace('-II.csv', '').replace('-I.csv', '')
    
symbols = sorted(set(symbols))
symbols

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    df = pd.DataFrame()
    sum1 = 0
    print(symbol)
    for i in tqdm(hyphens):
        print(i)
        file_path = input_path + symbol + i + '.csv'
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Ticker'] = np.where(df['Option_Type']=='XX', symbol + 'Fut_Monthly' + ' - ' + i[1:],
                                df['Ticker'])
        file_path = new_close_path + '_' + symbol + i + '_adj.csv.csv'
        if os.path.isfile(file_path):
            new_close_df = pd.read_csv(file_path)
            
            new_close_df['Date'] = pd.to_datetime(new_close_df['Date'], dayfirst=True)
            
            new_close_df['Remainder'] = new_close_df['Adj_strike'] % new_close_df['Adj_strike'].astype(int)
            
            new_close_df['Ticker'] = np.where(new_close_df['Remainder']==0, 
                            symbol + new_close_df['Adj_strike'].astype(int).astype(str) + new_close_df['OPTION_TYP'] + ' - ' + i[1:],
                            symbol + new_close_df['Adj_strike'].astype(float).astype(str) + new_close_df['OPTION_TYP'] + ' - ' + i[1:])            
            
            new_close_df['Ticker'] = np.where(new_close_df['OPTION_TYP']=='XX', symbol + 'Fut_Monthly' + ' - ' + i[1:], 
                                              new_close_df['Ticker'])
            
            df = df.drop(['OPT_Close'], axis=1)
            new_close_df = new_close_df.drop(['Remainder'], axis=1)
            df = df.merge(new_close_df[['Date', 'Ticker', 'new_close']], on=['Date', 'Ticker'], how='left')
            
            
            df = df.rename(columns={'new_close' : 'OPT_Close'})
            df = df[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI', 
                     'Strike', 'Option_Type', 'EXPIRY_DT', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega',  
                     'EQ_Open',  'EQ_High',   'EQ_Low',  'EQ_Close',  'At_The_Money',  'Delta_5_Strike',  
                     'Delta_10_Strike', 'Delta_15_Strike',  'Delta_20_Strike',  'Delta_25_Strike',  
                     'Delta_30_Strike', 'Delta_35_Strike', 'Delta_40_Strike',  'Delta_45_Strike', 'Delta_50_Strike']]
            df.to_csv(output_path + symbol + i + '.csv' ,index=False)
        